In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 경고 메시지가 뜨지 않게 설정
import warnings 
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 인코딩
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# 검정
from scipy import stats
# 로지스틱 모델 만들기
from statsmodels.formula.api import logit

# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [2]:
df = pd.read_parquet('open/concat/2018_회원정보.parquet')

FileNotFoundError: [Errno 2] No such file or directory: 'open/concat/2018_회원정보.parquet'

In [ ]:
df

### 의미없는 컬럼 제거

In [ ]:
import statsmodels
print(statsmodels.__version__)

In [ ]:
# 데이터의 도수의 종류가 1개인 데이터를 변수에 담는다.
cols_same_value = df.columns[df.nunique() == 1]

In [ ]:
df.drop(list(cols_same_value), axis = 1, inplace = True)

### 결측치를 가진 컬럼 리스트에 담기

In [ ]:
na_box = []

for idx, val in df.isna().sum().items() :
    # print(f'{idx}의 결측치 : {val}')
    if val > 0 :
        na_box.append(idx)

In [ ]:
na_box

- test 데이터의 Segment는 모두 결측치로 되어있다.
- 다른 칼럼의 경우 결측치 비율에 따라 처리한다.

In [ ]:
df[na_box].isna().sum() / df.shape[0]

### 결측 데이터 처리

1. 가입통신회사코드

In [ ]:
df['가입통신회사코드'].value_counts(normalize = True, dropna = False)

In [ ]:
# 최빈값 대체
df.fillna({'가입통신회사코드' : df['가입통신회사코드'].mode()[0]}, inplace = True)

In [ ]:
df['가입통신회사코드'].isna().sum()

2. 직장시도명

In [ ]:
df['직장시도명'].value_counts(normalize = True, dropna = False)

In [ ]:
# 직장시도명과 거주시도명을 비교해 본다.
a1 = df.loc[(df['직장시도명'].notna()) & (df['직장시도명'] == df['거주시도명'])].shape[0]
print(round(a1 / sum(df['직장시도명'].notna()), 3))

- 직장시도명 컬럼이 결측치가 아닌 데이터 중 96.4% 정도가 일치한다.
- 직장시도명 컬럼은 삭제한다.

In [ ]:
df.drop('직장시도명', axis = 1, inplace = True)

3. _1순위신용체크구분
- 해당 컬럼의 결측치는 미비한 수준이니 대표값으로 단순대체한다.

In [ ]:
df['_1순위신용체크구분'].value_counts()

In [ ]:
df.fillna({'_1순위신용체크구분' : df['_1순위신용체크구분'].mode()[0]}, inplace = True)

In [ ]:
df['_1순위신용체크구분'].isna().sum()

4. _2순위신용체크구분

In [ ]:
df['_2순위신용체크구분'].value_counts(normalize = True, dropna = False)

In [ ]:
a1 = sum((df['_2순위신용체크구분'].notna()) & (df['_2순위신용체크구분'] == df['_1순위신용체크구분']))
print(a1 / sum(df['_2순위신용체크구분'].notna()))

In [ ]:
ct1 = pd.crosstab(df.loc[df['_2순위신용체크구분'].notna(), '_2순위신용체크구분'],
                 df.loc[df['_2순위신용체크구분'].notna(), '_1순위신용체크구분'],
                 normalize = 'index',
                 margins = True)

sns.heatmap(ct1, annot = True, fmt = '.2%', cmap = 'Blues')
plt.show()

- 데이터의 분포에서 결측치의 비율이 가장 크다.
- 두 분포의 비율이 비슷하다.
- _2순위신용체크구분은 삭제한다.

In [ ]:
df.drop('_2순위신용체크구분', axis = 1, inplace = True)

In [ ]:
na_box = []

for idx, val in df.isna().sum().items() :
    # print(f'{idx}의 결측치 : {val}')
    if val > 0 :
        na_box.append(idx)

In [ ]:
na_box

In [ ]:
df[na_box]

In [ ]:
cat_cols = ['연령', '가입통신회사코드', '거주시도명', '_1순위신용체크구분', '연회비발생카드수_B0M', 'Life_Stage']
for col in cat_cols :
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [ ]:
df.info()

In [ ]:
df.to_csv('data/회원정보_전처리(fin).csv', index = False)